## ETL Project: USA HOSPITALS
#### Dependencies and Setup

In [ ]:
import requests
import psycopg2
import pandas as pd
import config as creds
import sqlalchemy as sqlalchemy_package

#### Extracting Data via API interaction 
* ESRI Dataset: Definitive Healthcare: USA Hospital Beds

In [8]:
# Query URL
url = f"https://services7.arcgis.com/LXCny1HyhQCUSueu/arcgis/rest/services/Definitive_Healthcare_USA_Hospital_Beds/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"

In [9]:
# Get Hospitals JSON data
response = requests.get(url).json()
hospitals_data = response['features']
hospitals_data[1]['attributes']

{'OBJECTID': 2,
 'HOSPITAL_NAME': 'Southern Arizona VA Health Care System',
 'HOSPITAL_TYPE': 'VA Hospital',
 'HQ_ADDRESS': '3601 S 6th Ave',
 'HQ_ADDRESS1': None,
 'HQ_CITY': 'Tucson',
 'HQ_STATE': 'AZ',
 'HQ_ZIP_CODE': '85723',
 'COUNTY_NAME': 'Pima',
 'STATE_NAME': 'Arizona',
 'STATE_FIPS': '04',
 'CNTY_FIPS': '019',
 'FIPS': '04019',
 'NUM_LICENSED_BEDS': 295,
 'NUM_STAFFED_BEDS': 295,
 'NUM_ICU_BEDS': 2,
 'ADULT_ICU_BEDS': 2,
 'PEDI_ICU_BEDS': None,
 'BED_UTILIZATION': None,
 'Potential_Increase_In_Bed_Capac': 0,
 'AVG_VENTILATOR_USAGE': 2}

In [10]:
# Create Lists of Data Dictionaries
hospitals = []
hospitals_beds = []
hospitals_location = []
hospitals_geometry = []
number = 1
for record in hospitals_data:
    hospital_id = str(number) + "-" + record['attributes']['FIPS']
    number += 1
    # create hospitals dataset
    row = {}
    row['id'] = hospital_id
    row['hospital_name'] = record['attributes']['HOSPITAL_NAME']
    hospitals.append(row)
    # create hospitals beds dataset
    row = {}
    row['id'] = hospital_id
    row['licensed_beds'] = record['attributes']['NUM_LICENSED_BEDS']
    row['icu_beds'] = record['attributes']['NUM_ICU_BEDS']
    row['ventilator_usage'] = record['attributes']['AVG_VENTILATOR_USAGE']
    hospitals_beds.append(row)
    # create hospitals location dataset
    row = {}
    row['id'] = hospital_id
    row['hospital_type'] = record['attributes']['HOSPITAL_TYPE']
    row['hospital_hq_address'] = record['attributes']['HQ_ADDRESS']
    row['hospital_hq_city'] = record['attributes']['HQ_CITY']
    row['hospital_hq_state'] = record['attributes']['HQ_STATE']
    row['zip_code'] = record['attributes']['HQ_ZIP_CODE']
    row['county_name'] = record['attributes']['COUNTY_NAME']
    row['state_name'] = record['attributes']['STATE_NAME']
    hospitals_location.append(row)
    # create hospitals geometry
    row = {}
    row['id'] = hospital_id
    row['lat'] = record['geometry']['x']
    row['lng'] = record['geometry']['y']
    hospitals_geometry.append(row)    

#### Transformation Data
* Converting lists of dictionaries into DataFrames

In [11]:
# Creating DataFrame for Hospitals 
hospitals_geometry_pd = pd.DataFrame(hospitals_geometry)
hospitals_geometry_pd.set_index("id", inplace=True)
hospitals_geometry_pd.head()

,lat,lng
id,,
1-04013,-112.066157,33.495498
2-04019,-110.965885,32.181263
3-06019,-119.779742,36.773324
4-09009,-72.957610,41.284400
5-10003,-75.606532,39.740206


In [12]:
# Creating DataFrame for Hospitals Location Data
hospitals_location_pd = pd.DataFrame(hospitals_location)
hospitals_location_pd.set_index("id", inplace=True)
hospitals_location_pd.head()

,hospital_type,hospital_hq_address,hospital_hq_city,hospital_hq_state,zip_code,county_name,state_name
id,,,,,,,
1-04013,VA Hospital,650 E Indian School Rd,Phoenix,AZ,85012,Maricopa,Arizona
2-04019,VA Hospital,3601 S 6th Ave,Tucson,AZ,85723,Pima,Arizona
3-06019,VA Hospital,2615 E Clinton Ave,Fresno,CA,93703,Fresno,California
4-09009,VA Hospital,950 Campbell Ave,West Haven,CT,6516,New Haven,Connecticut
5-10003,VA Hospital,1601 Kirkwood Hwy,Wilmington,DE,19805,New Castle,Delaware


In [13]:
# Creating DataFrame for Hospitals 
hospitals_beds_pd = pd.DataFrame(hospitals_beds)
hospitals_beds_pd.set_index("id", inplace=True)
hospitals_beds_pd.head()

,licensed_beds,icu_beds,ventilator_usage
id,,,
1-04013,62.0,0,0
2-04019,295.0,2,2
3-06019,54.0,2,2
4-09009,216.0,1,2
5-10003,62.0,0,1


In [14]:
# Creating DataFrame for Hospitals 
hospitals_pd = pd.DataFrame(hospitals)
hospitals_pd.set_index("id", inplace=True)
hospitals_pd

,hospital_name
id,
1-04013,Phoenix VA Health Care System (AKA Carl T Hayd...
2-04019,Southern Arizona VA Health Care System
3-06019,VA Central California Health Care System
4-09009,VA Connecticut Healthcare System - West Haven ...
5-10003,Wilmington VA Medical Center
...,...
1996-27047,Mayo Clinic Hospital - Albert Lea
1997-27049,Mayo Clinic Hospital - Red Wing (FKA Fairview ...
1998-27049,Mayo Clinic Hospital - Cannon Falls


In [15]:
# Check Number of Unique Hospitals  
hospitals_pd_list = hospitals_pd['hospital_name'].unique()
len(hospitals_pd_list)

1960

### Create database connection

In [17]:
# Create PostgreSQL database connection using credentials form config.sys
database_url = f"postgres://{creds.PGUSER}:{creds.PGPASSWORD}@{creds.PGHOST}:5432/{creds.PGDATABASE}"
engine = sqlalchemy_package.create_engine(database_url,
                connect_args={'options': '-csearch_path={}'.format(creds.DBSCHEMA)})

In [18]:
# Confirm tables
engine.table_names()

['hospitals_insuarance',
 'hospitals_staff',
 'hospitals_finance',
 'us_hospitals',
 'hospitals_beds',
 'hospitals_location',
 'hospitals_geometry',
 'hospitals_info']

### Load DataFrames into database

In [ ]:
# Load US hospitals list
hospitals_pd.to_sql(name='us_hospitals', con=engine, if_exists='append', index=True)

In [ ]:
# Load US hospitals beds
hospitals_beds_pd.to_sql(name='hospitals_beds', con=engine, if_exists='append', index=True)

In [ ]:
# Load US hospitals location
hospitals_location_pd.to_sql(name='hospitals_location', con=engine, if_exists='append', index=True)

In [ ]:
# Load US hospitals geometry
hospitals_geometry_pd.to_sql(name='hospitals_geometry', con=engine, if_exists='append', index=True)

### Initial Query Analysis

In [6]:
def load_data(schema='public', table='us_hospitals', query=None):
    if query == None:
        sql_command = "SELECT * FROM {}.{};".format(str(schema), str(table))
    else: sql_command = query
    print ('Query:',sql_command)
    # Load the data
    data = pd.read_sql(sql_command, conn)
    print('Data shape:',data.shape)
    return (data)

In [ ]:
# Set up a connection to the postgres server via database driver
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
+" password="+ creds.PGPASSWORD
conn=psycopg2.connect(conn_string)

In [19]:
# Create hospitals request for California
hospitals_request = """
    SELECT h.id, h.hospital_name, l.state_name, l.county_name, l.zip_code, l.hospital_type, b.licensed_beds
    FROM us_hospitals AS h
    INNER JOIN hospitals_beds AS b USING (id)
    INNER JOIN hospitals_location AS l USING (id)
    WHERE l.state_name = 'California'
    ORDER BY b.licensed_beds DESC;
    """
hosp_request_result = load_data(query=hospitals_request)
hosp_request_result.set_index("id", inplace=True)
hosp_request_result = hosp_request_result.sort_values('hospital_name')
hosp_request_result

Query: 
    SELECT h.id, h.hospital_name, l.state_name, l.county_name, l.zip_code, l.hospital_type, b.licensed_beds
    FROM us_hospitals AS h
    INNER JOIN hospitals_beds AS b USING (id)
    INNER JOIN hospitals_location AS l USING (id)
    WHERE l.state_name = 'California'
    ORDER BY b.licensed_beds DESC;
    
Data shape: (322, 7)


,hospital_name,state_name,county_name,zip_code,hospital_type,licensed_beds
id,,,,,,
417-06059,AHMC Anaheim Regional Medical Center,California,Orange,92801,Short Term Acute Care Hospital,223.0
313-06029,Adventist Health Bakersfield (FKA San Joaquin ...,California,Kern,93301,Short Term Acute Care Hospital,254.0
316-06033,Adventist Health Clear Lake (FKA St Helena Hos...,California,Lake,95422,Critical Access Hospital,32.0
349-06037,Adventist Health Glendale (FKA Glendale Advent...,California,Los Angeles,91206,Short Term Acute Care Hospital,415.0
315-06031,Adventist Health Hanford,California,Kings,93230,Short Term Acute Care Hospital,173.0
...,...,...,...,...,...,...
370-06037,West Los Angeles Medical Center,California,Los Angeles,90034,Short Term Acute Care Hospital,265.0
372-06037,Whittier Hospital Medical Center,California,Los Angeles,90605,Short Term Acute Care Hospital,156.0
391-06037,Woodland Hills Medical Center,California,Los Angeles,91367,Short Term Acute Care Hospital,274.0


### Adding New Datasets for Analysis (Secondary ETL Process)
* Connection as a GROUPUSER with a GROUPPASSWORD 

In [31]:
# Set up a connection to the postgres server via database driver using group password
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.GROUPUSER \
+" password="+ creds.GROUPPASSWORD
conn=psycopg2.connect(conn_string)

In [21]:
# Read Hospitals General Information
hospitals_info = pd.read_csv('Resources/Hospital_General_Information.csv')
hospitals_info = hospitals_info.filter(['Facility ID', 'Facility Name', 'State', 'ZIP Code', 'Hospital Ownership'])
hospitals_info = hospitals_info.rename(columns={
                                        'Facility ID': 'facility_id', 'Facility Name': 'hospital_name',
                                        'State':'state_name', 'ZIP Code': 'zip_code', 'Hospital Ownership': 'ownership'})
hospitals_info.set_index("facility_id", inplace=True)
hospitals_info = hospitals_info.sort_values('hospital_name')
hospitals_info.head()

,hospital_name,state_name,zip_code,ownership
facility_id,,,,
49005F,1st Medical Group (Langley AFB),VA,23665,Department of Defense
05015F,60th Medical Group (Travis AFB),CA,94533,Department of Defense
02013F,673rd Medical Group (Joint Base Elmendorf-Rich...,AK,99506,Department of Defense
25039F,81st Medical Group (Keesler AFB),MS,39530,Department of Defense
36006F,88th Medical Group (Wright-Patterson AFB),OH,45433,Department of Defense


In [22]:
# Read Hospitals Profitability Information
hospitals_finance = pd.read_csv('Resources/hospital-profitability-2009-2013-.csv')
hospitals_finance = hospitals_finance.filter(['Facility Number', 'Facility Name', 'Year', 'County Name',
                                              'Income Statement Amount', 'Amount per Adjusted Patient Day'])
hospitals_finance = hospitals_finance.rename(columns={
                                        'Facility Number': 'facility_num', 'Facility Name': 'hospital_name',
                                        'Year': 'year', 'County Name':'county_name', 
                                        'Income Statement Amount': 'total_income',
                                        'Amount per Adjusted Patient Day': 'adj_patient_income'})
hospitals_finance.set_index("facility_num", inplace=True)
hospitals_finance = hospitals_finance.sort_values('hospital_name')
hospitals_finance

,hospital_name,year,county_name,total_income,adj_patient_income
facility_num,,,,,
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2012,Kings,2.200000e+00,NaN
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,1.283870e+06,12.0
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,0.000000e+00,0.0
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,0.000000e+00,0.0
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,-2.931670e+05,-3.0
...,...,...,...,...,...
NaN,NaN,2013,Statewide,1.857423e+09,63.0
NaN,NaN,2013,Statewide,5.090389e+09,171.0
NaN,NaN,2013,Statewide,9.385780e+08,32.0


In [23]:
# Read Hospitals Staff Information
hospitals_staff = pd.read_csv('Resources/hospital-staffing-2009-2013-.csv')
hospitals_staff = hospitals_staff.filter(['Facility Number', 'Facility Name', 'Year', 'County Name',
                                              'Hours Type', 'Productive Hours', 'Productive Hours per Adjusted Patient Day'])
hospitals_staff = hospitals_staff.rename(columns={
                                        'Facility Number': 'facility_num', 'Facility Name': 'hospital_name',
                                        'Year': 'year', 'County Name':'county_name', 
                                        'Hours Type': 'hours_type', 'Productive Hours': 'prod_hours',
                                        'Productive Hours per Adjusted Patient Day': 'adj_patient_hours'})
hospitals_staff.set_index("facility_num", inplace=True)
hospitals_staff = hospitals_staff.sort_values('hospital_name')
hospitals_staff

,hospital_name,year,county_name,hours_type,prod_hours,adj_patient_hours
facility_num,,,,,,
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2012,Kings,Technician & Specialist,523880,4.99
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,Management & Supervision,178507,1.65
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,Technician & Specialist,554785,5.12
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,Registered Nurse,674942,6.23
106164029.0,ADVENTIST MEDICAL CENTER - HANFORD,2011,Kings,Licensed Vocational Nurse,24406,0.23
...,...,...,...,...,...,...
NaN,NaN,2013,Statewide,Ancillary Cost Centers,178050752,5.99
NaN,NaN,2013,Statewide,Education Cost Centers,17710831,0.60
NaN,NaN,2013,Statewide,General Services Cost Centers,136680589,4.60


In [24]:
# Read Hospitals Insuarance Info
hospitals_insuarance = pd.read_csv('Resources/healthcare_census.csv')
hospitals_insuarance = hospitals_insuarance.filter(['Provider Id', 'Provider Name', 'Provider Zip Code',
                                              'Average Covered Charges', 'Average Total Payments'])
hospitals_insuarance = hospitals_insuarance.rename(columns={
                                        'Provider Id': 'provider_id', 'Provider Name': 'hospital_name',
                                        'Provider Zip Code': 'zip_code', 'Average Covered Charges': 'avg_cov_charges',
                                        'Average Total Payments': 'avg_tot_payments'})
hospitals_insuarance.set_index("provider_id", inplace=True)
hospitals_insuarance = hospitals_insuarance.sort_values('hospital_name')
hospitals_insuarance.head()

,hospital_name,zip_code,avg_cov_charges,avg_tot_payments
provider_id,,,,
190034,ABBEVILLE GENERAL HOSPITAL,70510,"13,733.08","4,414.91"
190034,ABBEVILLE GENERAL HOSPITAL,70510,"27,433.11","11,295.05"
190034,ABBEVILLE GENERAL HOSPITAL,70510,"13,101.01","4,828.46"
190034,ABBEVILLE GENERAL HOSPITAL,70510,"16,700.65","6,544.50"
190034,ABBEVILLE GENERAL HOSPITAL,70510,"16,317.28","7,015.07"


In [32]:
# Confirm tables
engine.table_names()

['hospitals_insuarance',
 'hospitals_staff',
 'hospitals_finance',
 'us_hospitals',
 'hospitals_beds',
 'hospitals_location',
 'hospitals_geometry',
 'hospitals_info']

#### Normalization
* NPI (National Provider Identifier) - unique identification number for health care providers
* Hospital Name
* Composite key: Zip code + address

In [30]:
# Check Hospitals List Inconsistency for California
esri_list = hosp_request_result['hospital_name'].unique().tolist()
esri_upper_list = []
[esri_upper_list.append(i.upper()) for i in esri_list]
staff_list = hospitals_staff['hospital_name'].unique().tolist()
hopsitals_list = []
[hopsitals_list.append(name)for name in staff_list if name in esri_upper_list]
len(hopsitals_list)

132

In [27]:
esri_upper_list[:20]

['AHMC ANAHEIM REGIONAL MEDICAL CENTER',
 'ADVENTIST HEALTH BAKERSFIELD (FKA SAN JOAQUIN COMMUNITY HOSPITAL)',
 'ADVENTIST HEALTH CLEAR LAKE (FKA ST HELENA HOSPITAL - CLEAR LAKE)',
 'ADVENTIST HEALTH GLENDALE (FKA GLENDALE ADVENTIST MEDICAL CENTER)',
 'ADVENTIST HEALTH HANFORD',
 'ADVENTIST HEALTH HOWARD MEMORIAL (FKA FRANK R HOWARD MEMORIAL HOSPITAL)',
 'ADVENTIST HEALTH LODI MEMORIAL',
 'ADVENTIST HEALTH REEDLEY',
 'ADVENTIST HEALTH SIMI VALLEY (FKA SIMI VALLEY HOSPITAL & HEALTH CARE SERVICES)',
 'ADVENTIST HEALTH SONORA (FKA SONORA REGIONAL MEDICAL CENTER)',
 'ADVENTIST HEALTH ST HELENA (FKA ST HELENA HOSPITAL - NAPA VALLEY)',
 'ADVENTIST HEALTH TEHACHAPI VALLEY (FKA TEHACHAPI HOSPITAL)',
 'ADVENTIST HEALTH TULARE (FKA TULARE REGIONAL MEDICAL CENTER)',
 'ADVENTIST HEALTH UKIAH VALLEY (FKA UKIAH VALLEY MEDICAL CENTER)',
 'ADVENTIST HEALTH WHITE MEMORIAL (FKA WHITE MEMORIAL MEDICAL CENTER)',
 'ALAMEDA HOSPITAL',
 'ALHAMBRA HOSPITAL MEDICAL CENTER',
 'ALTA BATES SUMMIT MEDICAL CENTER -

In [28]:
staff_list[:10]

['ADVENTIST MEDICAL CENTER - HANFORD',
 'ADVENTIST MEDICAL CENTER - REEDLEY',
 'AGNEWS STATE HOSPITAL',
 'AHMC ANAHEIM REGIONAL MEDICAL CENTER',
 'ALAMEDA COUNTY MEDICAL CENTER',
 'ALAMEDA HOSPITAL',
 'ALHAMBRA HOSPITAL',
 'ALTA BATES SUMMIT MED CTR-ALTA BATES CAMPUS',
 'ALTA BATES SUMMIT MED CTR-SUMMIT CAMPUS-HAWTHORNE',
 'ALVARADO HOSPITAL']

* Overall, normalization requires more time and resources
* Keep looking for alternative solutions

#### Database Updating Grop Work

In [ ]:
# Load US hospitals info
# hospitals_info.to_sql(name='hospitals_info', con=engine, if_exists='append', index=True)

In [ ]:
# Load US hospitals_staff
# hospitals_insuarance.to_sql(name='hospitals_insuarance', con=engine, if_exists='append', index=True)